# 5강) BERT를 활용한 Dense Passage Retrieval 실습

### Requirements

In [1]:
import torch
torch.set_printoptions(threshold=10_000)

In [2]:
# !pip install datasets
# !pip install transformers

## 데이터셋 로딩


KorQuAD train 데이터셋을 학습 데이터로 활용

In [1]:
from datasets import load_dataset, load_from_disk

# dataset = load_dataset("squad_kor_v1")
dataset = load_from_disk('../../data/train_dataset')

## 토크나이저 준비 - Huggingface 제공 tokenizer 이용

BERT를 encoder로 사용하므로, hugginface에서 제공하는 "bert-base-multilingual-cased" tokenizer를 활용

In [2]:
from transformers import AutoTokenizer
import numpy as np

# model_checkpoint = "bert-base-multilingual-cased"
# model_checkpoint = "klue/roberta-base"

model_checkpoint = 'Huffon/sentence-klue-roberta-base'

# model_checkpoint = "klue/bert-base"
# model_checkpoint = "xlm-roberta-base"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")


In [3]:
tokenizer

PreTrainedTokenizerFast(name_or_path='Huffon/sentence-klue-roberta-base', vocab_size=32000, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [4]:
tokenized_input = tokenizer(dataset['train'][0]['context'], padding="max_length", truncation=True, max_length=510)
tokenizer.decode(tokenized_input['input_ids'])

'[CLS] 미국 상의원 또는 미국 상원 ( United States Senate ) 은 양원제인 미국 의회의 상원이다. [UNK] n [UNK] n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1 / 3씩 상원의원을 새로 선출하여 연방에 보낸다. [UNK] n [UNK] n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할 ( 하원의 법안을 거부할 권한 등 ) 을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션 ( 공공건강보험기관 ) 의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다. 날짜 = 2017 - 02 - 05 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

## Dense encoder (BERT) 학습 시키기

HuggingFace BERT를 활용하여 question encoder, passage encoder 학습

In [5]:
from tqdm import tqdm, trange
import argparse
import random
import torch
import torch.nn.functional as F
from transformers import BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup, RobertaForSequenceClassification

torch.manual_seed(2021)
torch.cuda.manual_seed(2021)
np.random.seed(2021)
random.seed(2021)

1) Training Dataset 준비하기 (question, passage pairs)

---



In [6]:
# Use subset (128 example) of original training dataset 
# sample_idx = np.random.choice(range(len(dataset['train'])), 128)
# training_dataset = dataset['train'][sample_idx]

training_dataset = dataset['train']

In [7]:
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)

q_seqs = tokenizer(training_dataset['question'], padding="max_length", truncation=True, return_tensors='pt', return_token_type_ids=False, max_length=510)
p_seqs = tokenizer(training_dataset['context'], padding="max_length", truncation=True, return_tensors='pt', return_token_type_ids=False, max_length=510)
# q_seqs = tokenizer(training_dataset['question'], padding="max_length", truncation=True, return_tensors='pt')
# p_seqs = tokenizer(training_dataset['context'], padding="max_length", truncation=True, return_tensors='pt')


In [8]:
# train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'],
#                         q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'],)

train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'],
                        q_seqs['input_ids'], q_seqs['attention_mask'])

2) BERT encoder 학습시키기

BertEncoder 모델 정의 후, question encoder, passage encoder에 pre-trained weight 불러오기

In [9]:
from transformers import RobertaPreTrainedModel, RobertaModel

In [10]:
class RobertaEncoder(RobertaPreTrainedModel):
# class BertEncoder(BertPreTrainedModel):  
  def __init__(self, config):
    super().__init__(config)

    self.roberta = RobertaModel(config)
    # self.bert = BertModel(config)
    self.init_weights()
      
  def forward(self, input_ids, 
              attention_mask=None, token_type_ids=None): 
  
      outputs = self.roberta(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids #roberta시 주석
                          )
      
      pooled_output = outputs[1]

      return pooled_output


In [11]:
# load pre-trained model on cuda (if available)
p_encoder = RobertaEncoder.from_pretrained(model_checkpoint)
q_encoder = RobertaEncoder.from_pretrained(model_checkpoint)

# p_encoder = RobertaForSequenceClassification.from_pretrained(model_checkpoint)
# q_encoder = RobertaForSequenceClassification.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
  p_encoder.cuda()
  q_encoder.cuda()

In [12]:
# q_encoder

Train function 정의 후, 두개의 encoder fine-tuning 하기 (In-batch negative 활용) 


In [15]:
def train(args, dataset, p_model, q_model):
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
    {"params": [p for n, p in p_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
    {"params": [p for n, p in p_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    {"params": [p for n, p in q_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
    {"params": [p for n, p in q_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0}
  ]

  optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=args.learning_rate,
    eps=args.adam_epsilon
  )
  
  # Dataloader
  train_sampler = RandomSampler(dataset)
  train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=args.per_device_train_batch_size)

# tt
  t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)

  # Start training!
  global_step = 0
  
  p_model.zero_grad()
  q_model.zero_grad()
  torch.cuda.empty_cache()
  
  train_iterator = trange(int(args.num_train_epochs), desc="Epoch")

  for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration")

    for step, batch in enumerate(epoch_iterator):
      q_encoder.train()
      p_encoder.train()
      
      if torch.cuda.is_available():
        batch = tuple(t.cuda() for t in batch)

      p_inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  # 'token_type_ids': batch[2] # roberta시 주석
                  }
      
      q_inputs = {'input_ids': batch[2],
                  'attention_mask': batch[3],
                  # 'token_type_ids': batch[5] # roberta시 주석
                  }
      
      p_outputs = p_model(**p_inputs)  # (batch_size, emb_dim)
      q_outputs = q_model(**q_inputs)  # (batch_size, emb_dim)


      # Calculate similarity score & loss
      sim_scores = torch.matmul(q_outputs, torch.transpose(p_outputs, 0, 1))  # (batch_size, emb_dim) x (emb_dim, batch_size) = (batch_size, batch_size)
      
      # print('q_outputs: ',q_outputs)
      # print('p_outputs: ',p_outputs)
      # print('sim_scores: ',sim_scores)
    
      # target: position of positive samples = diagonal element 
      targets = torch.arange(0, args.per_device_train_batch_size).long()
      if torch.cuda.is_available():
        targets = targets.to('cuda')

      sim_scores = F.log_softmax(sim_scores, dim=1)

      loss = F.nll_loss(sim_scores, targets) 
      print(loss)

      loss.backward()
      optimizer.step()
      scheduler.step()
      q_model.zero_grad()
      p_model.zero_grad()
      global_step += 1
      
      torch.cuda.empty_cache()


    
  return p_model, q_model




In [16]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01
)


In [17]:
from tqdm import tqdm

In [ ]:
p_encoder, q_encoder = tqdm(train(args, train_dataset, p_encoder, q_encoder))

In [19]:
p_encoder.save_pretrained('../encoders/p_encoder')
q_encoder.save_pretrained('../encoders/q_encoder')

In [58]:
p_encoder.from_pretrained('../encoders/p_encoder')
q_encoder.from_pretrained('../encoders/q_encoder')

You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at ../encoders/p_encoder were not used when initializing RobertaEncoder: ['bert.encoder.layer.10.attention.output.dense.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.5.attention.output.dense.bias', 'bert.encoder.layer.0.output.dense.bias', 'bert.encoder.layer.7.attention.output.dense.bias', 'bert.encoder.layer.7.attention.self.key.weight', 'bert.encoder.layer.8.attention.output.dense.bias', 'bert.encoder.layer.5.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.8.attention.self.query.bias', 'bert.encoder.layer.11.attention.self.query.weight', 'bert.encoder.layer.7.output.dense.weight', 'bert.encoder.layer.7.attention.self.value.weight', 'bert.encoder.layer.11.output.dense.bias', 'bert.encoder.layer.2.output.LayerNo

RobertaEncoder(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768, padding_idx=0)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((7

In [47]:
p_encoder = RobertaEncoder.from_pretrained(model_checkpoint).to('cuda')
q_encoder = RobertaEncoder.from_pretrained(model_checkpoint).to('cuda')

## Dense Embedding을 활용하여 passage retrieval 실습해보기

In [59]:

valid_corpus = list(set([example['context'] for example in dataset['validation']]))#[:10]
sample_idx = random.choice(range(len(dataset['validation'])))
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

if not ground_truth in valid_corpus:
  valid_corpus.append(ground_truth)

print(query)
print(ground_truth, '\n\n')

# valid_corpus

루이 14세의 왕비 마리아 테래사는 어느 나라 공주인가?
예부터 노트르담 다리는 큰 다리(Grand-pont)라고 불리면서 센 강을 가로지르는 교통 수단의 역할을 하였다. 이후 886년 노르만족이 파리 지역을 침공했을 당시 구조가 파괴되어 다시 지어졌다. 다시 지어진 다리는 밀브레 다리라고 불리기도 하였다. 1406년 대홍수 동안 다리는 다시 유실된다.\n\n1412년 5월 3일 샤를 6세가 부지에 다리의 골격을 새로 정비하고 최초로 노트르담이라는 이름을 하사한다. 그가 정비토록 지시한 다리의 구조는 견고한 목재를 통해 쌩마르탱 가와 다른 곳을 연결하도록 되어 있었다. 이때 다리 건축에만 7년이 소요되었으며 양옆으로는 각각 30여 가구가 있었다고 한다. 그러나 이 다리는 1499년 10월 25일 아침 9시경 지반 침하와 정비 부실로 붕괴된다.\n\n석재로의 다리 건축이 같은 해 시작되었지만 당분간 주민들은 연락선을 타고 센 강을 건너다녔다. 이 시기에는 아치 형으로 된 석재 다리가 지어졌으며 이탈리아 출신의 건축가이자 철학자였던 프라 지오반니가 건축을 맡았다. 그의 건축은 1507년에 완공되었으며 여전히 당시의 60여 개 벽돌과 석재는 보존되고 있다. 이후 상권의 중심으로 떠오른 다리 인근은 사람들이 모이는 곳으로 성장했다.\n\n1660년 노트르담 다리는 스페인 펠리페 4세의 딸이었던 마리아 테레사가 프랑스의 루이 14세의 왕비로 발탁되어 파리로 들어올 당시 최초의 다리가 되는 영예를 안기도 한다. 1646~1788년 동안 다리 인근의 가옥이 모두 도시 정비의 일환으로 파괴되었다.\n\n 1853년 새로운 석재 구조로 기존의 돌다리를 덮었다. 하지만 기존의 예술적 아름다움은 상당부분 경감되고 만다. 새로운 다리는 아치형으로 지어진 것이었지만 작은 흠이 있었다. 그것은 건축 보수 이후 연락선이 지나다니다 빈번히 사고가 난 것이었다. 때문에 1891년에서 1910년 사이 너무도 잦은 사고 탓에 사람들은 이곳을 악마의 다리라고 부르기도 했다고 한다. 때문에 센 강

앞서 학습한 passage encoder, question encoder을 이용해 dense embedding 생성

In [60]:
def to_cuda(batch):
  return tuple(t.cuda() for t in batch)

In [61]:
with torch.no_grad():
  p_encoder.eval()
  q_encoder.eval()

  # q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
  q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt', max_length=510).to('cuda')
  q_emb = q_encoder(**q_seqs_val).to('cpu')  #(num_query, emb_dim)

  p_embs = []
  for p in valid_corpus:
    p = tokenizer(p, padding="max_length", truncation=True, return_tensors='pt', max_length=510).to('cuda')
    p_emb = p_encoder(**p).to('cpu').numpy()
    p_embs.append(p_emb)

p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)

print(p_embs.size(), q_emb.size())

torch.Size([235, 768]) torch.Size([1, 768])


생성된 embedding에 dot product를 수행 => Document들의 similarity ranking을 구함

In [62]:
dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
print(dot_prod_scores.size())

rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
# print(dot_prod_scores)
# print(rank)

torch.Size([1, 235])


Top-5개의 passage를 retrieve 하고 ground truth와 비교하기

In [63]:
k = 5
print("[Search query]\n", query, "\n")
print("[Ground truth passage]")
print(ground_truth, "\n")

for i in range(k):
  print("Top-%d passage with score %.4f" % (i+1, dot_prod_scores.squeeze()[rank[i]]))
  print(valid_corpus[rank[i]])

[Search query]
 루이 14세의 왕비 마리아 테래사는 어느 나라 공주인가? 

[Ground truth passage]
예부터 노트르담 다리는 큰 다리(Grand-pont)라고 불리면서 센 강을 가로지르는 교통 수단의 역할을 하였다. 이후 886년 노르만족이 파리 지역을 침공했을 당시 구조가 파괴되어 다시 지어졌다. 다시 지어진 다리는 밀브레 다리라고 불리기도 하였다. 1406년 대홍수 동안 다리는 다시 유실된다.\n\n1412년 5월 3일 샤를 6세가 부지에 다리의 골격을 새로 정비하고 최초로 노트르담이라는 이름을 하사한다. 그가 정비토록 지시한 다리의 구조는 견고한 목재를 통해 쌩마르탱 가와 다른 곳을 연결하도록 되어 있었다. 이때 다리 건축에만 7년이 소요되었으며 양옆으로는 각각 30여 가구가 있었다고 한다. 그러나 이 다리는 1499년 10월 25일 아침 9시경 지반 침하와 정비 부실로 붕괴된다.\n\n석재로의 다리 건축이 같은 해 시작되었지만 당분간 주민들은 연락선을 타고 센 강을 건너다녔다. 이 시기에는 아치 형으로 된 석재 다리가 지어졌으며 이탈리아 출신의 건축가이자 철학자였던 프라 지오반니가 건축을 맡았다. 그의 건축은 1507년에 완공되었으며 여전히 당시의 60여 개 벽돌과 석재는 보존되고 있다. 이후 상권의 중심으로 떠오른 다리 인근은 사람들이 모이는 곳으로 성장했다.\n\n1660년 노트르담 다리는 스페인 펠리페 4세의 딸이었던 마리아 테레사가 프랑스의 루이 14세의 왕비로 발탁되어 파리로 들어올 당시 최초의 다리가 되는 영예를 안기도 한다. 1646~1788년 동안 다리 인근의 가옥이 모두 도시 정비의 일환으로 파괴되었다.\n\n 1853년 새로운 석재 구조로 기존의 돌다리를 덮었다. 하지만 기존의 예술적 아름다움은 상당부분 경감되고 만다. 새로운 다리는 아치형으로 지어진 것이었지만 작은 흠이 있었다. 그것은 건축 보수 이후 연락선이 지나다니다 빈번히 사고가 난 것이었다. 때문에 1891년에서 1910년 사이 너무도 잦은 사고 